In [2]:
!pip install geopy

     |████████████████████████████████| 118 kB 6.2 MB/s eta 0:00:01


In [3]:
from geopy.geocoders import Nominatim
import pandas as pd

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql import HiveContext

hive_context = HiveContext(sc)

spark = SparkSession \
    .builder \
    .appName("Teste API") \
    .enableHiveSupport() \
    .getOrCreate()

geo = spark.read.orc('/datalake/dadosbrutos/olist_geolocation_dataset.orc')

In [5]:
coord_array = [[str(row.geolocation_lat), str(row.geolocation_lng)] 
             for row in geo.select('geolocation_lat', 'geolocation_lng').distinct().collect()]

In [6]:
def extrai_cep(coord, counter):
    print(f"{counter}º chamada, coordenadas: {coord}", end ="\r")
    
    geolocator = Nominatim(user_agent="test_app", timeout=15)
    location = geolocator.reverse(coord) 
    try:
        return({"cep":location.raw['address']['postcode'], 
                "lat":coord[0], 
                "lng":coord[1]})
    except:
        pass

In [7]:
location_list = []
limit = 2000
counter = 0
none_results = 0 
import time

start = time.time()

for item in coord_array:
    cep = extrai_cep(item, counter)
    if cep == None:
        none_results += 1
    else:
        location_list.append(cep)
        counter += 1
    if counter == limit: break
        
df = pd.DataFrame(location_list)
df = df.drop_duplicates()
df.head(100)

end = time.time()
tempo = (end - start)/60
print(f"\nTempo: {tempo} minutos")
print(f"Nao encontrados: {none_results}")

KeyboardInterrupt: 

In [13]:
df.head(100)

,cep,lat,lng
0,12280-015,-23.086135110739242,-45.689912093946575
1,12246-140,-23.217447205118503,-45.90768709991921
2,12236-160,-23.248120899830894,-45.89728759763259
3,12231-140,-23.21138053566277,-45.88524183374691
4,12232-841,-23.28743618235325,-45.89000442062436
...,...,...,...
95,13219-630,-23.170269614328607,-46.84436622055807
96,13224-430,-23.215001079922637,-46.83430924939157
97,13211-534,-23.20905371798737,-46.90959302567946
98,13271570,-22.987338658148907,-47.012867692334005


In [14]:
df.count()

cep    2000
lat    2000
lng    2000
dtype: int64